<a href="https://colab.research.google.com/github/PolinaKudryavtseva/third-year-homeworks/blob/main/dz1_polinakudryavtseva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Домашнее задание 1. Работа Полины Кудрявцевой**

Краулер собирает отзывы с сайта afisha.ru, из раздела про драматические театральные постановки. Отзывы там поделены на положительные и отрицательные, эта программа обкачивает каждый раздел по отдельности. Положительных и отрицательных комментариев собрано по 50, тестовые отзывы собираются отдельно.

Библиотеки импортируются по необходимости.

In [1]:
import requests
from pprint import pprint
import time

session = requests.session()

for _ in range(5):
    response = session.get('https://www.afisha.ru/msk/schedule_theatre/drama_plays/').text
    time.sleep(3)

In [2]:
from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(response,'html.parser')
positive_links = []
negative_links = []

#на главной страницы раздела про драматические постановки собираем страницы каждого спектакля,
#добавляем к ним часть адреса на положительные/отрицательные отзывы

plays = soup.find_all('h3')
for play in plays:
  link = re.search('/performance/\d*/', str(play))
  positive_link = 'https://www.afisha.ru' + str(link.group()) + 'reviews/positive/'
  positive_links.append(positive_link)
  negative_link = 'https://www.afisha.ru' + str(link.group()) + 'reviews/negative/'
  negative_links.append(negative_link)

In [27]:
positive_texts = [] #здесь будут лежать необработанные тексты положительных отзывов
while len(positive_texts) <= 50:
  for link in positive_links:
    response = session.get(link).text
    time.sleep(2)
    soup = BeautifulSoup(response,'html.parser')
    texts = soup.find_all('div', {'class':'restrict-text review__text'})
    for i in texts:
      text = i.find('p')
      positive_texts.append(text)

In [29]:
negative_texts = [] #здесь будут лежать необработанные тексты отрицательных отзывов
while len(negative_texts) <= 50:
  for link in negative_links:
    response = session.get(link).text
    time.sleep(2)
    soup = BeautifulSoup(response,'html.parser')
    texts = soup.find_all('div', {'class':'restrict-text review__text'})
    for i in texts:
      text = i.find('p')
      negative_texts.append(text)

In [5]:
import nltk
from nltk.tokenize import word_tokenize

In [6]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Тексты по отдельности положительных и отрицательных отзывов приводятся к нижнему регистру и лемматизируются, затем приводятся к начальной форме.

In [43]:
positive_lemmas = []
for text in positive_texts:
  text = str(text)
  text = re.sub('<\w+>', '', text)
  text = re.sub('<//\w+>', '', text)
  tokens = word_tokenize(text)
  positive_tokens = [w for w in tokens if w.isalpha()]
  for word in positive_tokens:
      analise = morph.parse(word)
      one = analise[0]
      form = one.normal_form
      positive_lemmas.append(form)

In [44]:
negative_lemmas = []
for text in negative_texts:
  text = str(text)
  text = re.sub('<\w+>', '', text)
  text = re.sub('<//\w+>', '', text)
  tokens = word_tokenize(text)
  negative_tokens = [w for w in tokens if w.isalpha()]
  for word in negative_tokens:
      analise = morph.parse(word)
      one = analise[0]
      form = one.normal_form
      negative_lemmas.append(form)

In [50]:
from collections import Counter

In [64]:
positive_freqlist = Counter() #здесь будут словари частотностей уникальных (тех, которые есть только в таких отзывах) слов положительных отзывов
negative_freqlist = Counter() #здесь будут словари частотностей уникальных слов отрицательных отзывов

for word in positive_lemmas:
  if word not in negative_lemmas:
    positive_freqlist[word] += 1
for word in negative_lemmas:
  if word not in positive_lemmas:
    negative_freqlist[word] += 1

positive_dictionary = dict(positive_freqlist)
negative_dictionary = dict(negative_freqlist)

In [89]:
test_positive_texts = [] #здесь идет работа с тестовой подборкой положительных отзывов
response = session.get('https://www.afisha.ru/performance/83857/reviews/positive/').text
soup = BeautifulSoup(response,'html.parser')
texts = soup.find_all('div', {'class':'restrict-text review__text'})
for i in texts:
  text = i.find('p')
  test_positive_texts.append(text)

In [90]:
test_positive_lemmas_list = []
for text in test_positive_texts:
  text = str(text)
  text = re.sub('<\w+>', '', text)
  text = re.sub('<//\w+>', '', text)
  tokens = word_tokenize(text)
  test_positive_tokens = [w for w in tokens if w.isalpha()]
  test_positive_lemmas = []
  for word in test_positive_tokens:
      analise = morph.parse(word)
      one = analise[0]
      form = one.normal_form
      test_positive_lemmas.append(form)
  test_positive_lemmas_list.append(test_positive_lemmas)

In [66]:
test_negative_texts = [] #здесь идет работа с тестовой подборкой отрицательных отзывов
response = session.get('https://www.afisha.ru/performance/86394/reviews/negative/').text
soup = BeautifulSoup(response,'html.parser')
texts = soup.find_all('div', {'class':'restrict-text review__text'})
for i in texts:
  text = i.find('p')
  test_negative_texts.append(text)

In [102]:
test_negative_lemmas_list = []
for text in test_negative_texts:
  text = str(text)
  text = re.sub('<\w+>', '', text)
  text = re.sub('<//\w+>', '', text)
  tokens = word_tokenize(text)
  test_negative_tokens = [w for w in tokens if w.isalpha()]
  test_negative_lemmas = []
  for word in test_negative_tokens:
      analise = morph.parse(word)
      one = analise[0]
      form = one.normal_form
      test_negative_lemmas.append(form)
  test_negative_lemmas_list.append(test_negative_lemmas)

Следующая функция делает предположение, к какому виду отзывов относится предлагаемый текст. Потом считается accuracy.

In [83]:
from sklearn.metrics import accuracy_score

In [103]:
def review_detect(positive_dictionary, negative_dictionary, test_lemmas, test_negative_lemmas, test_positive_lemmas):

  positiveness = 0
  negativeness = 0

  answers = []
  original = []

  for text in test_lemmas:
    if text in test_positive_lemmas:
      original.append('positive')
    if text in test_negative_lemmas:
      original.append('negative')

    for lemma in text:
      if lemma in positive_dictionary.keys():
        positiveness += 1
      if lemma in negative_dictionary.keys():
        negativeness += 1
    if positiveness >= negativeness:
      answers.append('positive')
    else:
      answers.append('negative')

  accuracy = print("Accuracy: %.4f" % accuracy_score(answers, original))

  return accuracy

In [104]:
print('Проверим, что скажет функция про позитивный комментарий.')
review_detect(positive_dictionary, negative_dictionary, test_positive_lemmas_list, test_negative_lemmas_list, test_positive_lemmas_list)

Проверим, что скажет функция про позитивный комментарий.
Accuracy: 1.0000


In [106]:
print('Проверим, что скажет функция про негативный комментарий.')
review_detect(positive_dictionary, negative_dictionary, test_negative_lemmas_list, test_negative_lemmas_list, test_positive_lemmas_list)

Проверим, что скажет функция про негативный комментарий.
Accuracy: 0.0000


Предложения:
- accuracy будет намного выше, если сделать подборку больше. Такой краулер будет работать дольше, но слов в словарях получится больше и вероятность правильного предсказания повысится
- можно было бы добавить сюда что-то вроде словаря синонимов, или word2vec, чтобы если в отзыве нет конкретного слова, но есть близкое к слову из словаря по значению, этот отзыв имел более правильное предсказание.